<a href="https://colab.research.google.com/github/RheaBhalekar24/ML/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=801e85fcde595f0534f1a012fb440de6e69985635b2cddbd0b904d3b78161e7e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pandas as pd
import pyspark
print(pyspark.__version__)
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import SparkSession

3.5.0


In [ ]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

In [ ]:
# df2 = spark.read.csv('sentiment_train.csv')
df3 = spark.read.csv('Table_1.csv')
df3.show(truncate = False, n = 4)
print(df3.type)


+-------------------------+-----------------------+
|_c0                      |_c1                    |
+-------------------------+-----------------------+
|<b></b>                  |<b>n, %, or mean±SD</b>|
|Age (years)              |45.8 (±16.2)           |
|Educational level (years)|5.9 (±4.0)             |
|Sex-female               |56 (49.1%)             |
+-------------------------+-----------------------+
only showing top 4 rows



AttributeError: 'DataFrame' object has no attribute 'type'

In [ ]:
df2.show(truncate = False , n =10 )


+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|_c0|_c1       |_c2                         |_c3     |_c4            |_c5                                                                                                                |
+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|0  |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
|0  |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |
|0  |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY|mattycus   

In [ ]:
data = df2.select('_c5',col('_c0').cast("Int").alias("label"))

In [ ]:
data.show(truncate = False,n = 10)

+-------------------------------------------------------------------------------------------------------------------+-----+
|_c5                                                                                                                |label|
+-------------------------------------------------------------------------------------------------------------------+-----+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|0    |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |0    |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                          |0    |
|my whole body feels itchy and like its on fire                                                                     |0    |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.     |0    |
|@Kwesid

In [ ]:
# train test split
split = data.randomSplit([0.7,0.3])
train = split[0]
test = split[1]

In [ ]:
test=split[1].withColumnRenamed("label","true_label")
test.show()

+--------------------+----------+
|                 _c5|true_label|
+--------------------+----------+
|           FUCK YOU!|         0|
|     I'll get on ...|         0|
|   No man is wort...|         0|
|   i miss you guy...|         0|
|             over it|         0|
|  so much for a g...|         0|
|  still no phone....|         0|
| - Why must peopl...|         0|
| .... Feeling pai...|         0|
| ..y everything i...|         0|
| 6........tomorro...|         0|
| :S this tweetdec...|         0|
| ;( noooo! why? t...|         0|
| @beeeebzy KYS if...|         0|
| @kvetchingeditor...|         0|
| AAAARRRRRGGGGGHH...|         0|
| AWW I GOTA DO HO...|         0|
| A_ just got the ...|         0|
| Alone and invisi...|         0|
| Cardboard doesn'...|         0|
+--------------------+----------+
only showing top 20 rows



In [ ]:
train_rows=train.count()
test_rows=test.count()

print("Total train :",train_rows)
print("Total test :", test_rows)

Total train : 59257
Total test : 25408


In [ ]:
# tokenizing
tokenizer = Tokenizer(inputCol="_c5", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(train)
tokenizedTrain.show(truncate=False, n=5)

+-----------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------+
|_c5                                                                                            |label|SentimentWords                                                                                                          |
+-----------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------+
|          i want some ben&amp;jerrys cake batter please ugh                                    |0    |[, , , , , , , , , , i, want, some, ben&amp;jerrys, cake, batter, please, ugh]                                          |
|     #canucks                                                                                  |0  

In [ ]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(),
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=10)

+----------------------------------------------------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|_c5                                                                                                                                     |label|SentimentWords                                                                                                                                                         |MeaningfulWords                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------------------------+-----+--

In [ ]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrain = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrain.show(truncate=False, n=3)

+-----+---------------------------------------------------------------------+----------------------------------------------------------------------------------------+
|label|MeaningfulWords                                                      |features                                                                                |
+-----+---------------------------------------------------------------------+----------------------------------------------------------------------------------------+
|0    |[, , , , , , , , , , want, ben&amp;jerrys, cake, batter, please, ugh]|(262144,[13007,39485,166368,190256,218824,249180,255800],[1.0,1.0,1.0,1.0,1.0,10.0,1.0])|
|0    |[, , , , , #canucks]                                                 |(262144,[113038,249180],[1.0,5.0])                                                      |
|0    |[, , , , , ok, thats, win.]                                          |(262144,[10345,188292,198197,249180],[1.0,1.0,1.0,5.0])                                 

In [ ]:
#Training Model
lr = LogisticRegression(labelCol="label", featuresCol="features",
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrain)
print ("Training Done")

Training Done


In [ ]:
#Prepare Testing data
tokenizedTest = tokenizer.transform(test)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest)
numericTest.show(truncate=False, n=2)

+-------------------------------+----------+-------------------------------------------+--------------------------------+-------------------------------------------------------+
|_c5                            |true_label|SentimentWords                             |MeaningfulWords                 |features                                               |
+-------------------------------+----------+-------------------------------------------+--------------------------------+-------------------------------------------------------+
|          FUCK YOU!            |0         |[, , , , , , , , , , fuck, you!]           |[, , , , , , , , , , fuck, you!]|(262144,[164046,237111,249180],[1.0,1.0,10.0])         |
|     I'll get on it right away!|0         |[, , , , , i'll, get, on, it, right, away!]|[, , , , , get, right, away!]   |(262144,[42645,229166,249180,252722],[1.0,1.0,5.0,1.0])|
+-------------------------------+----------+-------------------------------------------+----------------------

In [ ]:
#Prediction
raw_prediction = model.transform(numericTest)
raw_prediction.printSchema()

root
 |-- _c5: string (nullable = true)
 |-- true_label: integer (nullable = true)
 |-- SentimentWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MeaningfulWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
Final_prediction = raw_prediction.select("MeaningfulWords", "prediction", "true_label")
Final_prediction.show(n=4, truncate = False)

+------------------------------------------------------------------------------------------------------------+----------+----------+
|MeaningfulWords                                                                                             |prediction|true_label|
+------------------------------------------------------------------------------------------------------------+----------+----------+
|[, , , , , , , , , , fuck, you!]                                                                            |0.0       |0         |
|[, , , , , get, right, away!]                                                                               |0.0       |0         |
|[, , , man, worth, tears,, one, make, cry!]                                                                 |0.0       |0         |
|[, , , miss, guys, , , , , think, wearing, skinny, jeans, cute, sweater, heels, , , really, sure, , , today]|0.0       |0         |
+--------------------------------------------------------------------

In [ ]:
Total_True=Final_prediction.filter(Final_prediction['prediction']==Final_prediction['true_label']).count()
Alldata=Final_prediction.count()
Accuracy=Total_True/Alldata
print("Accuracy Score is:", Accuracy*98, '%')

Accuracy Score is: 98.0 %
